In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [ ]:
#Preprocessing the data
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #Tokenizing each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        #Adding documents in the corpus
        documents.append((w, intent['tag']))

        #Adding to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#Lemmatizing, lowering each word and removnig duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#Sorting classes
classes = sorted(list(set(classes)))

#Documents = conbination between patterns and intents
print(len(documents), "documents")

#Classes = intents
print(len(classes), "classes", classes)

#Words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
#Creating the training data
training = []

#Creating an empty array for our output
output_empty = [0] * len(classes)

#Training set, bag of words for each sentence
for doc in documents:
    #Initializing the bag of words
    bag = []
    #List of tokonized words for the pattern
    pattern_words = doc[0]
    #Lemmatizing each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #Creating the bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

#Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

#Creating train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

In [ ]:
#Building the mode
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

#Compiling the model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")